In [1]:
import pandas as pd

In [2]:
def standardize_headers(df, func=None):
    df.columns = df.columns.str.replace(' ', '_').str.lower()
    if func:
      df = df.apply(func)
    return df

def remove_high_na_columns(df, threshold=0.3):
    """
    Removes columns from a DataFrame if more than `threshold` proportion of values are NA or 0.
    
    Parameters:
    df (pd.DataFrame): The input DataFrame.
    threshold (float): The proportion of missing (or zero) values above which a column is removed.
    
    Returns:
    pd.DataFrame: The cleaned DataFrame.
    """
    # Treat 0s as NA
    df_replaced = df.replace(0, pd.NA)
    
    # Compute the proportion of missing values per column
    missing_ratio = df_replaced.isna().mean()
    
    # Keep columns where the missing ratio is below the threshold
    return df.loc[:, missing_ratio <= threshold]

In [3]:
country_indicators_raw = pd.read_csv("data/environment_pop_data_all.csv")
country_indicators_raw = standardize_headers(country_indicators_raw)
country_indicators_long = pd.melt(country_indicators_raw, id_vars= ['country_code', 'indicator_name', 'indicator_code'], var_name = 'year')
country_indicators_long[['year', 'value']] = country_indicators_long[['year', 'value']].apply(pd.to_numeric, errors='coerce')
country_indicators_long = country_indicators_long[country_indicators_long['year'] > 2010]
country_indicators_long.drop(columns = ['indicator_name'], inplace=True)
country_indicators = country_indicators_long.pivot_table(index=['country_code', 'year'], columns='indicator_code', values='value').reset_index()
country_indicators = remove_high_na_columns(country_indicators)

In [4]:
climate_data = pd.read_csv("data/climate_data/ERA5_Monthly_Country_Weather_2010_2024.csv")

In [5]:
climate_data

,country_na,date,system:time_start,temperature_2m,total_precipitation_sum,potential_evaporation_sum
0,Chad,2010-01,1.262304e+12,295.270399,0.000029,-0.274852
1,Malawi,2010-01,1.262304e+12,296.915585,0.248372,-0.210955
2,Zambia,2010-01,1.262304e+12,295.522374,0.237523,-0.234153
3,Zimbabwe,2010-01,1.262304e+12,297.442308,0.137571,-0.270994
4,Burundi,2010-01,1.262304e+12,293.489225,0.242564,-0.201473
...,...,...,...,...,...,...
7735,Ecuador,2024-12,1.733011e+12,293.990356,0.243766,-0.252331
7736,Peru,2024-12,1.733011e+12,292.478298,0.254705,-0.287672
7737,El Salvador,2024-12,1.733011e+12,297.775743,0.025290,-0.384632
7738,Guatemala,2024-12,1.733011e+12,293.522251,0.107090,-0.244403


In [6]:
# extract year and month, and convert to numeric
climate_data = standardize_headers(climate_data)

climate_data['year'] = climate_data['date'].str.split('-').str[0].astype(int)
climate_data['month'] = climate_data['date'].str.split('-').str[1].astype(int)

climate_data.drop(columns=['date', 'system:time_start'], inplace=True)

In [7]:
inflation_data = pd.read_csv("data/economic/inflation_all_countries_sorted_cleaned.csv")

In [8]:
migration_data = pd.read_csv('data/migration/full_iom_dtm_data.csv')

migration_data['date'] = pd.to_datetime(migration_data['reportingDate'])
migration_data['month'] = migration_data['date'].dt.month
migration_data['year'] = migration_data['date'].dt.year

# Define a more descriptive column mapping for the columns we want to keep
column_mapping = {
    'admin0Name': 'country_name',  # Keep country name
    'admin0Pcode': 'country_code', # In case you need to map country codes to names
    'numPresentIdpInd': 'internally_displaced_persons',
    'date': 'date',
    'year': 'year',
    'month': 'month'
    
}

# List of columns to keep (everything else will be dropped)
columns_to_keep = list(column_mapping.keys())

# Keep only the columns we need
migration_data = migration_data[columns_to_keep]
# Rename the columns
migration_data.rename(columns=column_mapping, inplace=True)

total_idps_per_month = migration_data[['year', 'month', 'country_name','country_code', 'internally_displaced_persons']].groupby(['year', 'month', 'country_name', 'country_code']).sum().reset_index()

In [9]:
#Merge with country indicators

total_idps_per_month = pd.merge(total_idps_per_month, climate_data, how='left', 
                                left_on=['country_name', 'year', 'month'],
                                right_on=['country_na', 'year', 'month'])

In [10]:
climate_data.country_na.unique()

array(['Chad', 'Malawi', 'Zambia', 'Zimbabwe', 'Burundi', 'South Africa',
       'Niger', 'Sudan', 'Libya', 'Ethiopia', 'Djibouti', 'Somalia',
       'South Sudan', 'Kenya', 'Uganda', 'Mozambique', 'Madagascar',
       'Mali', 'Burkina Faso', 'Benin', 'Nigeria', 'Cameroon', 'Ukraine',
       'Mongolia', 'Afghanistan', 'Pakistan', 'Nepal', 'Sri Lanka',
       'Papua New Guinea', 'Fiji', 'Vanuatu', 'Indonesia', 'Iraq',
       'Lebanon', 'Yemen', 'Dominica', 'Grenada', 'Haiti', 'Ecuador',
       'Peru', 'El Salvador', 'Guatemala', 'Honduras'], dtype=object)

In [16]:
#fix the previous line

missing_countries = total_idps_per_month[total_idps_per_month['potential_evaporation_sum'].isna()]
missing_countries.country_name.unique()

array(['Philippines (the)', 'Democratic Republic of the Congo',
       'Antigua and Barbuda', 'Sri lanka', 'Central African Republic',
       'Bolivia (Plurinational State of)',
       "Lao People's Democratic Republic",
       'Democratic Republic of The Congo', 'Bahamas (the)',
       'Republic of Armenia', 'Saint Vincent and the Grenadines', 'Chad',
       'Lebanon', 'Sudan', 'Syrian Arab Republic'], dtype=object)

In [29]:
# Merge with climate data
total_idps_per_month = pd.merge(total_idps_per_month, climate_data, how='left', on=['country_code', 'year', 'month'])

KeyError: 'country_code'

In [20]:
# view and define emdat_cc_countries_expanded
file_path = 'data/climate_catastrophes/emdat_full.xlsx'
emdat_cc_expanded = pd.read_excel(file_path) 

In [21]:
# Step 3: Create a list of countries to filter by
target_countries = total_idps_per_month['country_name'].unique().tolist()

# Step 4: Filter for only those countries
emdat_cc_expanded = emdat_cc_expanded[emdat_cc_expanded['Country'].isin(target_countries)]

In [22]:
climate_catastrophes_data_subset = emdat_cc_expanded[['Country', 'Start Year', 'Start Month', 'End Year', 'End Month',  'Disaster Type', 'CPI', 'Total Affected']]
climate_catastrophes_data_subset["End Year"] = climate_catastrophes_data_subset["End Year"].fillna(climate_catastrophes_data_subset["Start Year"])
climate_catastrophes_data_subset["End Month"] = climate_catastrophes_data_subset["End Month"].fillna(climate_catastrophes_data_subset["Start Month"])

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_664\3030979710.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  climate_catastrophes_data_subset["End Year"] = climate_catastrophes_data_subset["End Year"].fillna(climate_catastrophes_data_subset["Start Year"])
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_664\3030979710.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  climate_catastrophes_data_subset["End Month"] = climate_catastrophes_data_subset["End Month"].fillna(climate_catastrophes_data_subset["Start 

In [23]:
climate_catastrophes_data_subset

,Country,Start Year,Start Month,End Year,End Month,Disaster Type,CPI,Total Affected
3,Burundi,2010,1.0,2010,1.0,Flood,71.563596,1000.0
4,Pakistan,2010,1.0,2010,1.0,Mass movement (wet),71.563596,26700.0
5,Haiti,2010,1.0,2010,1.0,Earthquake,71.563596,3700000.0
11,Bolivia (Plurinational State of),2010,1.0,2010,1.0,Flood,71.563596,227860.0
12,Indonesia,2010,1.0,2010,1.0,Flood,71.563596,28500.0
...,...,...,...,...,...,...,...,...
5907,Madagascar,2025,2.0,2025,3.0,Storm,NaN,50254.0
5908,Madagascar,2025,3.0,2025,3.0,Storm,NaN,4101.0
5909,Mozambique,2025,3.0,2025,3.0,Storm,NaN,385135.0
5910,Malawi,2025,3.0,2025,3.0,Storm,NaN,NaN


In [24]:
expanded_rows = []
for _, row in climate_catastrophes_data_subset.iterrows():
    for year in range(row["Start Year"], row["End Year"] + 1):  # Iterate through years
        # Handle potential NaN values in Start Month and End Month and ensure values are integers
        start_month = (
            int(row["Start Month"]) if year == row["Start Year"] and not pd.isna(row["Start Month"]) else 1
        )
        end_month = (
            int(row["End Month"]) if year == row["End Year"] and not pd.isna(row["End Month"]) else 12
        )
        
        # Iterate through months and create the expanded rows
        for m in range(start_month, end_month + 1):
            new_row = row.copy()
            new_row["month"] = m  # Assign the month
            new_row["year"] = year  # Assign the correct year
            expanded_rows.append(new_row)

In [25]:
climate_catastrophes_data = pd.DataFrame(expanded_rows)

In [26]:
climate_catastrophes_data.isna().sum()

Country             0
Start Year          0
Start Month       425
End Year            0
End Month         132
Disaster Type       0
CPI               258
Total Affected    289
month               0
year                0
dtype: int64

In [27]:
climate_catastrophes_data.head(10)

,Country,Start Year,Start Month,End Year,End Month,Disaster Type,CPI,Total Affected,month,year
3,Burundi,2010,1.0,2010,1.0,Flood,71.563596,1000.0,1,2010
4,Pakistan,2010,1.0,2010,1.0,Mass movement (wet),71.563596,26700.0,1,2010
5,Haiti,2010,1.0,2010,1.0,Earthquake,71.563596,3700000.0,1,2010
11,Bolivia (Plurinational State of),2010,1.0,2010,1.0,Flood,71.563596,227860.0,1,2010
12,Indonesia,2010,1.0,2010,1.0,Flood,71.563596,28500.0,1,2010
17,Indonesia,2010,1.0,2010,1.0,Flood,71.563596,355.0,1,2010
21,Afghanistan,2010,2.0,2010,2.0,Mass movement (wet),71.563596,130.0,2,2010
27,Pakistan,2010,2.0,2010,2.0,Flood,71.563596,NaN,2,2010
29,Ecuador,2010,1.0,2010,2.0,Flood,71.563596,500.0,1,2010
29,Ecuador,2010,1.0,2010,2.0,Flood,71.563596,500.0,2,2010


In [28]:
total_idps_per_month

,year,month,country_name,country_code,internally_displaced_persons,AG.LND.FRST.K2,AG.LND.PRCP.MM,AG.LND.TOTL.K2,AG.SRF.TOTL.K2,EG.CFT.ACCS.RU.ZS,...,EN.GHG.N2O.MT.CE.AR5,EN.GHG.N2O.PI.MT.CE.AR5,EN.GHG.N2O.TR.MT.CE.AR5,EN.GHG.N2O.WA.MT.CE.AR5,ER.FSH.AQUA.MT,ER.FSH.CAPT.MT,ER.FSH.PROD.MT,ER.H2O.FWTL.K3,ER.H2O.INTR.K3,SP.POP.TOTL
0,2010,6,Sudan,SDN,30933,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010,11,Haiti,HTI,2137764,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2011,1,Haiti,HTI,1612754,3752.74,1440.0,27560.0,27750.0,1.0,...,1.3652,17.0,141.000,792.000,600.0,16530.0,17130.0,1.45,13.007,9914904.0
3,2011,2,Sudan,SDN,98298,NaN,NaN,NaN,2505810.0,25.9,...,261.2350,32.0,1.029,5.197,2000.0,71008.0,73008.0,NaN,NaN,36140806.0
4,2011,3,Haiti,HTI,1360988,3752.74,1440.0,27560.0,27750.0,1.0,...,1.3652,17.0,141.000,792.000,600.0,16530.0,17130.0,1.45,13.007,9914904.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
971,2025,1,Lebanon,LBN,1333317,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
972,2025,1,Sudan,SDN,69461010,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
973,2025,2,Lebanon,LBN,1236267,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
974,2025,2,Syrian Arab Republic,SYR,20983938,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
# Step 4: Merge with emdat_cc_countries_expanded using the 'month', 'year', and 'Country' columns
merged_df = pd.merge(
    total_idps_per_month,
    climate_catastrophes_data,  # This is your climate data
    left_on=['year', 'month', 'country_name'],
    right_on=['year', 'month', 'Country'],
    how='left'          # Keep all climate data rows, even without IDP data
)
merged_df

,year,month,country_name,country_code,internally_displaced_persons,AG.LND.FRST.K2,AG.LND.PRCP.MM,AG.LND.TOTL.K2,AG.SRF.TOTL.K2,EG.CFT.ACCS.RU.ZS,...,ER.H2O.INTR.K3,SP.POP.TOTL,Country,Start Year,Start Month,End Year,End Month,Disaster Type,CPI,Total Affected
0,2010,6,Sudan,SDN,30933,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010,11,Haiti,HTI,2137764,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Haiti,2010.0,10.0,2011.0,12.0,Epidemic,71.563596,513997.0
2,2010,11,Haiti,HTI,2137764,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Haiti,2010.0,11.0,2010.0,11.0,Storm,71.563596,5020.0
3,2011,1,Haiti,HTI,1612754,3752.74,1440.0,27560.0,27750.0,1.0,...,13.007,9914904.0,Haiti,2010.0,10.0,2011.0,12.0,Epidemic,71.563596,513997.0
4,2011,2,Sudan,SDN,98298,NaN,NaN,NaN,2505810.0,25.9,...,NaN,36140806.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1139,2025,1,Lebanon,LBN,1333317,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1140,2025,1,Sudan,SDN,69461010,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Sudan,2024.0,8.0,2025.0,3.0,Epidemic,NaN,57447.0
1141,2025,2,Lebanon,LBN,1236267,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1142,2025,2,Syrian Arab Republic,SYR,20983938,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
#drop 'Country' column

merged_df.drop(columns=['Country'], inplace=True)

In [31]:
merged_df

,year,month,country_name,country_code,internally_displaced_persons,AG.LND.FRST.K2,AG.LND.PRCP.MM,AG.LND.TOTL.K2,AG.SRF.TOTL.K2,EG.CFT.ACCS.RU.ZS,...,ER.H2O.FWTL.K3,ER.H2O.INTR.K3,SP.POP.TOTL,Start Year,Start Month,End Year,End Month,Disaster Type,CPI,Total Affected
0,2010,6,Sudan,SDN,30933,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010,11,Haiti,HTI,2137764,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2010.0,10.0,2011.0,12.0,Epidemic,71.563596,513997.0
2,2010,11,Haiti,HTI,2137764,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2010.0,11.0,2010.0,11.0,Storm,71.563596,5020.0
3,2011,1,Haiti,HTI,1612754,3752.74,1440.0,27560.0,27750.0,1.0,...,1.45,13.007,9914904.0,2010.0,10.0,2011.0,12.0,Epidemic,71.563596,513997.0
4,2011,2,Sudan,SDN,98298,NaN,NaN,NaN,2505810.0,25.9,...,NaN,NaN,36140806.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1139,2025,1,Lebanon,LBN,1333317,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1140,2025,1,Sudan,SDN,69461010,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2024.0,8.0,2025.0,3.0,Epidemic,NaN,57447.0
1141,2025,2,Lebanon,LBN,1236267,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1142,2025,2,Syrian Arab Republic,SYR,20983938,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
# Create column 'climate_catastrophe' to indicate if a climate catastrophe occurred

merged_df['climate_catastrophe'] = merged_df['Disaster Type'].notnull().astype(int)
merged_df['climate_catastrophe'].value_counts()

climate_catastrophe
0    589
1    555
Name: count, dtype: int64

In [33]:
#count NAs per column
na_counts = merged_df.isna().sum()
print(na_counts)

year                              0
month                             0
country_name                      0
country_code                      0
internally_displaced_persons      0
AG.LND.FRST.K2                  316
AG.LND.PRCP.MM                  377
AG.LND.TOTL.K2                  293
AG.SRF.TOTL.K2                  287
EG.CFT.ACCS.RU.ZS               331
EG.CFT.ACCS.UR.ZS               331
EG.CFT.ACCS.ZS                  331
EG.EGY.PRIM.PP.KD               352
EG.ELC.ACCS.ZS                  287
EG.FEC.RNEW.ZS                  350
EN.GHG.ALL.MT.CE.AR5            589
EN.GHG.CH4.AG.MT.CE.AR5         330
EN.GHG.CH4.BU.MT.CE.AR5         311
EN.GHG.CH4.FE.MT.CE.AR5         447
EN.GHG.CH4.IC.MT.CE.AR5         311
EN.GHG.CH4.MT.CE.AR5            505
EN.GHG.CH4.PI.MT.CE.AR5         311
EN.GHG.CH4.TR.MT.CE.AR5         311
EN.GHG.CH4.WA.MT.CE.AR5         311
EN.GHG.CO2.BU.MT.CE.AR5         311
EN.GHG.CO2.IC.MT.CE.AR5         318
EN.GHG.CO2.IP.MT.CE.AR5         311
EN.GHG.CO2.LU.MT.CE.AR5     

In [34]:
# get unique values in the country_name with missingmess in CPI

missing_cpi = merged_df[merged_df['CPI'].isna()]['country_name'].unique()
missing_cpi
# we have some missingness in CPI, check if we can find a solution for this

array(['Sudan', 'Pakistan', 'Haiti', 'Philippines (the)', 'Mali',
       'South Sudan', 'Iraq', 'Vanuatu', 'Nigeria', 'Yemen', 'Nepal',
       'Burundi', 'Libya', 'Cameroon', 'Fiji', 'Ecuador',
       'Papua New Guinea', 'Democratic Republic of the Congo',
       'Afghanistan', 'Peru', 'Somalia', 'Dominica', 'Sri lanka',
       'Ethiopia', 'Central African Republic', 'Chad',
       'Bolivia (Plurinational State of)', 'Mongolia',
       'Democratic Republic of The Congo', 'Burkina Faso', 'Mozambique',
       'Bahamas (the)', 'Niger', 'El Salvador', 'Zimbabwe',
       'Republic of Armenia', 'Ukraine', 'Djibouti', 'Uganda', 'Lebanon',
       'Benin', 'Grenada', 'Saint Vincent and the Grenadines', 'Malawi',
       'Zambia', 'Syrian Arab Republic'], dtype=object)

In [35]:
# save it to an csv file
merged_df.to_csv('data/merged_climate_iom_data.csv', index=False)